In [2]:
from bs4 import BeautifulSoup
import requests
import sqlite3
from urllib.parse import urljoin
import time
import re
import json

In [ ]:

# Conectarse (crea el archivo si no existe)
conexion = sqlite3.connect("biblioteca.db")

# Activar claves foráneas
conexion.execute("PRAGMA foreign_keys = ON;")

cursor = conexion.cursor()

# Crear tablas
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Autor (
    id_autor INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS Genero (
    id_genero INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_genero TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS Libro (
    id_libro INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL,
    precio REAL,
    id_genero INTEGER,
    id_autor INTEGER,
    upc TEXT,
    disponibles INTEGER,
    rating INTEGER,
    link TEXT,
    FOREIGN KEY (id_genero) REFERENCES Genero(id_genero),
    FOREIGN KEY (id_autor) REFERENCES Autor(id_autor)
);
""")

conexion.commit()
conexion.close()

print("Base de datos 'biblioteca.db' creada correctamente.")


✅ Base de datos 'biblioteca.db' creada correctamente.


In [ ]:

CATALOGUE_URL = "http://books.toscrape.com/catalogue/"

# LIsta para guardar los libros
todos_los_libros = []

# Diccionario para encontrar la puntuación
rating_dict = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}

# Recorrer las páginas
for pagina in range(1, 51):
    try:
        print(f"Página {pagina}")
        
        url = (f"http://books.toscrape.com/catalogue/page-{pagina}.html")
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()

        soup = BeautifulSoup(resp.text, "html.parser")
        libros = soup.find_all("article", class_="product_pod")

        for libro in libros:
            try:
                # --- Título ---
                titulo = libro.h3.a["title"]

                # --- Precio ---
                precio_str = libro.find("p", class_="price_color").text
                precio_numero = re.findall(r"\d+\.\d+", precio_str)[0]
                precio = float(precio_numero)

                # --- Link completo ---
                link_relativo = libro.h3.a["href"]
                link_completo = urljoin(CATALOGUE_URL, link_relativo)

                # --- Género ---
                detalle_resp = requests.get(link_completo, timeout=10)
                detalle_resp.raise_for_status()
                sopa_detalle = BeautifulSoup(detalle_resp.text, "html.parser")
                breadcrumb = sopa_detalle.find("ul", class_="breadcrumb")
                genero = breadcrumb.find_all("a")[2].text.strip()

                # --- UPC ---
                tabla = sopa_detalle.find("table", class_="table table-striped")
                upc = tabla.find("tr").td.text.strip()

                # --- Disponibilidad ---
                disponibilidad_texto = tabla.find_all("tr")[5].td.text.strip()
                match = re.search(r"\d+", disponibilidad_texto)
                disponibles = int(match.group()) if match else 0

                # --- Puntuación ---
                rating_p = sopa_detalle.find("p", class_="star-rating")
                clases = rating_p.get("class", [])
                rating = 0
                for clase in clases:
                    if clase in rating_dict:
                        rating = rating_dict[clase]
                        break
                
                # --- Guardar libro ---
                todos_los_libros.append((titulo, precio, genero, link_completo, upc, disponibles, rating))

                time.sleep(0.2)

            except Exception as e:
                print(f"Error al procesar libro '{titulo}': {e}")
                break

    except Exception as e:
        print(f"Error en la página {pagina}: {e}")
        break

print(f"\nTotal de libros obtenidos: {len(todos_los_libros)}")


📄 Página 1
📄 Página 2
📄 Página 3
📄 Página 4
📄 Página 5
📄 Página 6
📄 Página 7
📄 Página 8
📄 Página 9
📄 Página 10
📄 Página 11
📄 Página 12
📄 Página 13
📄 Página 14
📄 Página 15
📄 Página 16
📄 Página 17
📄 Página 18
📄 Página 19
📄 Página 20
📄 Página 21
📄 Página 22
📄 Página 23
📄 Página 24
📄 Página 25
📄 Página 26
📄 Página 27
📄 Página 28
📄 Página 29
📄 Página 30
📄 Página 31
📄 Página 32
📄 Página 33
📄 Página 34
📄 Página 35
📄 Página 36
📄 Página 37
📄 Página 38
📄 Página 39
📄 Página 40
📄 Página 41
📄 Página 42
📄 Página 43
📄 Página 44
📄 Página 45
📄 Página 46
📄 Página 47
📄 Página 48
📄 Página 49
📄 Página 50

✅ Total de libros obtenidos: 1000


In [7]:
libros_dict = []
for libro in todos_los_libros:
    libros_dict.append({
        "titulo": libro[0],
        "precio": libro[1],
        "genero": libro[2],
        "link": libro[3],
        "upc": libro[4],
        "disponibles": libro[5],
        "rating": libro[6]
    })

# Guardar en archivo JSON
with open("libros_scrapeados.json", "w", encoding="utf-8") as f:
    json.dump(libros_dict, f, ensure_ascii=False, indent=4)

In [5]:
def buscar_autor_por_titulo(titulo):
    query = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{titulo}"
    resp = requests.get(query)
    data = resp.json()
    try:
        return data["items"][0]["volumeInfo"]["authors"][0]
    except (KeyError, IndexError):
        return "Desconocido"

with open("libros_scrapeados.json", "r", encoding="utf-8") as f:
    libros = json.load(f)
    
autores_dict = []

for libro in libros:
    titulo = libro["titulo"]
    autor = buscar_autor_por_titulo(titulo)
    
    autores_dict.append({
        "titulo": titulo,
        "autor": autor
    })

# Guardar autores en JSON
with open("autores.json", "w", encoding="utf-8") as f:
    json.dump(autores_dict, f, ensure_ascii=False, indent=4)

print(f"✅ Se guardaron {len(autores_dict)} autores en JSON")

✅ Se guardaron 1000 autores en JSON


In [ ]:
with open("libros_scrapeados.json", "r", encoding="utf-8") as f:
    libros = json.load(f)


conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()
conexion.execute("PRAGMA foreign_keys = ON;")

# Diccionarios para no duplicar inserciones
generos_cache = {}  # nombre_genero -> id_genero

# Insertar géneros
for libro in libros:
    genero = libro["genero"]
    if genero not in generos_cache:
        cursor.execute("INSERT OR IGNORE INTO Genero (nombre_genero) VALUES (?)", (genero,))
        cursor.execute("SELECT id_genero FROM Genero WHERE nombre_genero = ?", (genero,))
        generos_cache[genero] = cursor.fetchone()[0]

conexion.commit()
cursor.close()
conexion.close()


In [7]:
with open("autores.json", "r", encoding="utf-8") as f:
    autores_json = json.load(f)

conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()
conexion.execute("PRAGMA foreign_keys = ON;")  # asegura integridad referencial

# Diccionarios para no duplicar inserciones
autores_cache = {}  # nombre_autor -> id_autor

# Insertar autores
for item in autores_json:
    autor = item["autor"]
    if autor not in autores_cache:
        cursor.execute("INSERT OR IGNORE INTO Autor (nombre) VALUES (?)", (autor,))
        cursor.execute("SELECT id_autor FROM Autor WHERE nombre = ?", (autor,))
        autores_cache[autor] = cursor.fetchone()[0]

conexion.commit()
cursor.close()
conexion.close()

In [8]:
with open("libros_scrapeados.json", "r", encoding="utf-8") as f:
    libros = json.load(f)

with open("autores.json", "r", encoding="utf-8") as f:
    autores_json = json.load(f)

conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()
conexion.execute("PRAGMA foreign_keys = ON;")  # asegura integridad referencial

for libro, item_autor in zip(libros, autores_json):
    titulo = libro["titulo"]
    precio = libro["precio"]
    genero = libro["genero"]
    link = libro["link"]
    upc = libro["upc"]
    disponibles = libro["disponibles"]
    rating = libro["rating"]
    
    id_genero = generos_cache[genero]
    id_autor = autores_cache[item_autor["autor"]]

    cursor.execute("""
        INSERT INTO Libro
        (titulo, precio, id_genero, id_autor, upc, disponibles, rating, link)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (titulo, precio, id_genero, id_autor, upc, disponibles, rating, link))

conexion.commit()
cursor.close()
conexion.close()

print(f"✅ Se guardaron {len(libros)} libros.")

✅ Se guardaron 1000 libros.


In [8]:
conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()
num = -1
print("\nConsultando la cantidad de libros por rating:")
for rating in range(1, 6):
    cursor.execute(f"""
    SELECT
        Libro.titulo,
        Libro.precio,
        Genero.nombre_genero,
        Autor.nombre AS autor,
        Libro.rating
    FROM Libro
    JOIN Genero ON Libro.id_genero = Genero.id_genero
    JOIN Autor ON Libro.id_autor = Autor.id_autor
    WHERE Libro.rating = {rating}
    """)

    libros_rating_1 = cursor.fetchall()
    cantidad = len(libros_rating_1)
    print(f"Se recuperaron {cantidad} libros con rating {rating}.")
    if cantidad > num:
        num = cantidad
        rating_mayor = rating

print(f"\nEl rating con mayor cantidad de libros es {rating_mayor} con {num} libros.")
cursor.close()
conexion.close()


Consultando la cantidad de libros por rating:
Se recuperaron 226 libros con rating 1.
Se recuperaron 196 libros con rating 2.
Se recuperaron 203 libros con rating 3.
Se recuperaron 179 libros con rating 4.
Se recuperaron 196 libros con rating 5.

El rating con mayor cantidad de libros es 1 con 226 libros.


In [16]:
conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()

print("\nLibros con menos de 5 unidades disponibles")

cursor.execute("""
SELECT
    Libro.titulo,
    Libro.precio,
    Genero.nombre_genero,
    Autor.nombre AS autor,
    Libro.rating,
    Libro.disponibles
FROM Libro
JOIN Genero ON Libro.id_genero = Genero.id_genero
JOIN Autor ON Libro.id_autor = Autor.id_autor
WHERE Libro.disponibles < 5
""")

libros_pocos = cursor.fetchall()

print(f"Se recuperaron {len(libros_pocos)} libros con menos de 5 unidades disponibles:\n")
for libro in libros_pocos:
    print(libro)

cursor.close()
conexion.close()


Libros con menos de 5 unidades disponibles
Se recuperaron 355 libros con menos de 5 unidades disponibles:

('Vogue Colors A to Z: A Fashion Coloring Book', 52.35, 'Default', 'Valerie Steiker', 4, 4)
('The Shining (The Shining #1)', 27.88, 'Default', 'Stephen King', 3, 4)
("The Pilgrim's Progress", 50.26, 'Classics', 'John Bunyan', 2, 4)
('The Perfect Play (Play by Play #1)', 59.99, 'Romance', 'Jaci Burton', 3, 4)
('The Passion of Dolssa', 28.32, 'Historical Fiction', 'Julie Berry', 5, 4)
('The Jazz of Physics: The Secret Link Between Music and the Structure of the Universe', 38.71, 'Nonfiction', 'Stephon Alexander', 3, 4)
('The Hunger Games (The Hunger Games #1)', 29.85, 'Default', 'Suzanne Collins', 5, 4)
('The Hound of the Baskervilles (Sherlock Holmes #5)', 14.82, 'Classics', 'A. Conan Doyle', 2, 4)
('The Gunning of America: Business and the Making of American Gun Culture', 16.81, 'Nonfiction', 'Pamela Haag', 4, 4)
("The Geography of Bliss: One Grump's Search for the Happiest Place

In [20]:
conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()

print("\nAutores con más de 3 libros en la base de datos:")

cursor.execute("""
SELECT Autor.nombre, COUNT(Libro.id_libro) AS cantidad_libros
FROM Autor
JOIN Libro ON Autor.id_autor = Libro.id_autor
GROUP BY Autor.nombre
HAVING COUNT(Libro.id_libro) > 3
""")

resultado = cursor.fetchall()
for autor, cantidad in resultado:
    print(f"{autor}: {cantidad} libros")

cursor.close()
conexion.close()


Autores con más de 3 libros en la base de datos:
Bill Bryson: 5 libros
Cassandra Clare: 7 libros
David Sedaris: 4 libros
Desconocido: 26 libros
George R. R. Martin: 4 libros
J. K. Rowling: 6 libros
Maggie Stiefvater: 5 libros
Malcolm Gladwell: 4 libros
Natsuki Takaya: 8 libros
Sophie Kinsella: 6 libros
Stephen King: 11 libros


In [21]:
conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()

print("\nLibros con autores desconocidos")

cursor.execute("""
SELECT
    Libro.titulo,
    Genero.nombre_genero,
    Autor.nombre AS autor,
    Libro.rating,
    Libro.disponibles
FROM Libro
JOIN Genero ON Libro.id_genero = Genero.id_genero
JOIN Autor ON Libro.id_autor = Autor.id_autor
WHERE Autor.nombre = 'Desconocido'
""")

libros = cursor.fetchall()

print(f"Se recuperaron {len(libros)} libros.\n")
for libro in libros:
    print(libro)

cursor.close()
conexion.close()


Libros con autores desconocidos
Se recuperaron 26 libros.

('Olio', 'Poetry', 'Desconocido', 1, 19)
('Wall and Piece', 'Art', 'Desconocido', 4, 18)
('In a Dark, Dark Wood', 'Mystery', 'Desconocido', 1, 18)
('The Inefficiency Assassin: Time Management Tactics for Working Smarter, Not Longer', 'Default', 'Desconocido', 5, 16)
('Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Princess Jellyfish 2-in-1 Omnibus #1)', 'Sequential Art', 'Desconocido', 5, 16)
('The White Cat and the Monk: A Retelling of the Poem â\x80\x9cPangur BÃ¡nâ\x80\x9d', 'Childrens', 'Desconocido', 4, 15)
('The Project', 'Science Fiction', 'Desconocido', 1, 15)
('Done Rubbed Out (Reightman & Bailey #1)', 'Default', 'Desconocido', 5, 15)
('See America: A Celebration of Our National Parks & Treasured Sites', 'Travel', 'Desconocido', 3, 14)
("Don't Get Caught", 'Young Adult', 'Desconocido', 1, 14)
('Balloon Animals', 'Fiction', 'Desconocido', 3, 14)
('Rook', 'Add a comment', 'Desconocido', 4, 13)
('The Epidemic (The Program 0.

In [ ]:
conexion = sqlite3.connect("biblioteca.db")
cursor = conexion.cursor()

print("\nLibros por menos de 11 euros")

cursor.execute("""
SELECT
    Libro.titulo,
    Libro.precio,
    Genero.nombre_genero,
    Autor.nombre AS autor,
    Libro.rating,
    Libro.disponibles
FROM Libro
JOIN Genero ON Libro.id_genero = Genero.id_genero
JOIN Autor ON Libro.id_autor = Autor.id_autor
WHERE Libro.precio < 11.0
""")

libros = cursor.fetchall()

print(f"Se recuperaron {len(libros)} libros.\n")
for libro in libros:
    print(libro)

cursor.close()
conexion.close()


Libros con menos de 11 euros
Se recuperaron 23 libros.

('Patience', 10.16, 'Sequential Art', 'Guy de Maupassant', 3, 16)
('The Project', 10.65, 'Science Fiction', 'Desconocido', 1, 15)
('The Long Shadow of Small Ghosts: Murder and Memory in an American City', 10.97, 'Crime', 'Laura Tillman', 1, 15)
('The Argonauts', 10.93, 'Autobiography', 'Apollonius of Rhodes', 2, 15)
('Miss Peregrineâ\x80\x99s Home for Peculiar Children (Miss Peregrineâ\x80\x99s Peculiar Children #1)', 10.76, 'Default', 'Ransom Riggs', 1, 15)
('The Lucifer Effect: Understanding How Good People Turn Evil', 10.4, 'Psychology', 'Philip Zimbardo', 1, 14)
('Tastes Like Fear (DI Marnie Rome #3)', 10.69, 'Mystery', 'Sarah Hilary', 1, 14)
('Pet Sematary', 10.56, 'Horror', 'Stephen King', 3, 14)
('I Am Pilgrim (Pilgrim #1)', 10.6, 'Fiction', 'Terry Hayes', 4, 14)
('Greek Mythic History', 10.23, 'Default', 'History Brought Alive', 5, 14)
('Adulthood Is a Myth: A "Sarah\'s Scribbles" Collection', 10.9, 'Sequential Art', 'Sar

In [7]:
lista = []
diccionario = {}
for pagina in range(1, 2):  
    url = (f"https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-{pagina}.html")
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    libros = soup.find_all("article", class_="product_pod")
    for libro in libros:
        titulo = libro.h3.a.text

        libro.find("div", class_="product_price")
        precio_cadena = libro.find("p", class_="price_color").text
        precio_limpio = re.findall(r"\d+\.\d+", precio_cadena)[0]
        precio = float(precio_limpio)
        diccionario = {"titulo": titulo, "precio": precio}
        lista.append(diccionario)

print(lista)

[{'titulo': 'Tipping the Velvet', 'precio': 53.74}, {'titulo': 'Forever and Forever: The ...', 'precio': 29.69}, {'titulo': 'A Flight of Arrows ...', 'precio': 55.53}, {'titulo': 'The House by the ...', 'precio': 36.95}, {'titulo': 'Mrs. Houdini', 'precio': 30.25}, {'titulo': 'The Marriage of Opposites', 'precio': 28.08}, {'titulo': 'Glory over Everything: Beyond ...', 'precio': 45.84}, {'titulo': 'Love, Lies and Spies', 'precio': 20.55}, {'titulo': 'A Paris Apartment', 'precio': 39.01}, {'titulo': 'Lilac Girls', 'precio': 17.28}, {'titulo': 'The Constant Princess (The ...', 'precio': 16.62}, {'titulo': 'The Invention of Wings', 'precio': 37.34}, {'titulo': 'World Without End (The ...', 'precio': 32.97}, {'titulo': 'The Passion of Dolssa', 'precio': 28.32}, {'titulo': 'Girl With a Pearl ...', 'precio': 26.77}, {'titulo': 'Voyager (Outlander #3)', 'precio': 21.07}, {'titulo': 'The Red Tent', 'precio': 35.66}, {'titulo': 'The Last Painting of ...', 'precio': 55.55}, {'titulo': 'The Guern

In [11]:
lista = []

for pag in range(1,3):
    try:
        resp = requests.get(f"https://quotes.toscrape.com/page/{pag}/")
        resp.raise_for_status()

        soup = BeautifulSoup(resp.text, "html.parser")
        quotes = soup.find_all("div", class_="quote")
        for quote in quotes:
            texto = quote.find("span", class_="text").text
            autor = quote.find("small", class_="author").text
            lista.append({"texto": texto, "autor": autor})
    except Exception as e:
        print(f"Error en la página {pag}: {e}")
        break
print(lista)


[{'texto': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'autor': 'Albert Einstein'}, {'texto': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'autor': 'J.K. Rowling'}, {'texto': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'autor': 'Albert Einstein'}, {'texto': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'autor': 'Jane Austen'}, {'texto': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", 'autor': 'Marilyn Monroe'}, {'texto': '“Try not to become a man of success. Rather become a man of value.”', 'autor': 'Albert Einstein'}, {'texto': '“It is better to be hated for what you are than to be loved for what you are not.”', 'autor': 'André Gide'}, {'texto': "“I ha